In [8]:
import os
import sys
p = os.path.abspath('../..')
if p not in sys.path:
    sys.path.append(p)

from waveorder.io.writer import WaveorderWriter
import numpy as np
import zarr

# Writing data with no HCS Specification

### Initialize the writer

This will instantiate the writer class, no zarr stores, subfolders, etc have been created yet.  This step allows you to define which type of writer you want and let it know your directory to save the data

In [9]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop/Test', hcs=False, hcs_meta=None, verbose=True)

No existing directory found. Creating new directory at /Users/cameron.foltz/Desktop/Test


### Set Position and Create Zarr

'create_zarr_root()' will create an empty zarr store within the save directory.  No arrays have been added here.  Name of the store must be input by the user.

'create_position()' will create a subgroup in your zarr store under the first row and Nth column with the corresponding position number.  The current working directory of the writer is now this subfolder.  If you specify a name, the name will replace the default position subgroup name

In [10]:
writer.create_zarr_root('waveOrder_Writer_Example.zarr')

Creating new zarr store at /Users/cameron.foltz/Desktop/Test/waveOrder_Writer_Example.zarr


In [11]:
for i in range(10):
    writer.create_position(i, name=None)

Creating and opening subgroup Row_0/Col_0/Pos_000
Creating and opening subgroup Row_0/Col_1/Pos_001
Creating and opening subgroup Row_0/Col_2/Pos_002
Creating and opening subgroup Row_0/Col_3/Pos_003
Creating and opening subgroup Row_0/Col_4/Pos_004
Creating and opening subgroup Row_0/Col_5/Pos_005
Creating and opening subgroup Row_0/Col_6/Pos_006
Creating and opening subgroup Row_0/Col_7/Pos_007
Creating and opening subgroup Row_0/Col_8/Pos_008
Creating and opening subgroup Row_0/Col_9/Pos_009


### Check which position (subgroup) the writer is currently at

Useful function for when you are writing to different groups at different times.  i.e. 'position first' writing (example later)

In [13]:
print(f'Current store is at: {writer.get_current_pos_group()}')

Current store is at: <zarr.hierarchy.Group '/Row_0/Col_9/Pos_009'>


### Change to different position

if your group has a prefix, it must be specified

In [15]:
writer.open_position(0)

Opening subgroup Row_0/Col_0/Pos_000


### Create some fake data to save later

In [16]:
P = 10
T = 3
C = 2
Z = 65
Y = 256
X = 256

data = np.random.rand(P, T, C, Z, Y, X)

In [17]:
np.shape(data)

(10, 3, 2, 65, 256, 256)

### initialize the array which sets the data saving parameters

Here is where you will initialize the array in your current writer.store .  data_shape should match the (T, C, Z, Y, X) dimensions of your data.  Positions are separated by zarr subgroups under the root zarr store.

__chunk_size__ determines how zarr will chunk your data.  This means that when you later try to load the data, it will load one chunk at a time with this specified size.  To have the chunk be one z-slice, you would set chunk_size = (1,1,1,Y,X)

__chan_names__ describe the names of the channels of your data in the order in which they will be written.

__clims__ corresponds to the the display contrast limits in the metadata for every channel, if none, default values will be used

In [18]:
data_shape = (3,2,65,256,256)
chunk_size = (1,1,1,256,256)
chan_names = ['DAPI', 'Phase3D']
clims = [(0,1000),(-0.3, 0.3)]
dtype = 'float32'

In [20]:
writer.init_array(data_shape, chunk_size, dtype, chan_names, clims, overwrite=False)

### Write the data along specific dimensions

In [21]:
writer.write(data[0], t=[0,3], c=[0,2], z=[0,65])

# Complex, 'Position First' Acquisition

Let's say you have a dataset with 10 position with each position containing a data of size (T, C, Z, Y, X) = (3, 2, 65, 256, 256) and you want to save the first time point of every position first, and then move on to the next timepoint.  This will show you how to properly loop through this example case.

In [22]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop')
writer.create_zarr_root('waveOrder_Writer_Example_MultiPos.zarr')

Creating new zarr store at /Users/cameron.foltz/Desktop/waveOrder_Writer_Example_MultiPos.zarr


In [24]:
# Must first initialize all of the arrays, since you will be returning to them 
# at different times in the data-saving process.
num_pos = 10

for i in range(num_pos):
    writer.create_position(i)
    writer.init_array(data_shape, chunk_size, dtype, chan_names, clims, overwrite=False)    
# Now begin the rime loop 
for t in range(3):
    
    # At each time point, write data for all positions
    for pos in range(num_pos):
        
        print(f'Writing Position {pos} at Time {t}')
        
        #change to the correct position group
        writer.open_position(pos)
        
        #write this position data
        writer.write(data[pos, t], t=t, c=[0,2], z=[0,65])

Writing Position 0 at Time 0
Writing Position 1 at Time 0
Writing Position 2 at Time 0
Writing Position 3 at Time 0
Writing Position 4 at Time 0
Writing Position 5 at Time 0
Writing Position 6 at Time 0
Writing Position 7 at Time 0
Writing Position 8 at Time 0
Writing Position 9 at Time 0
Writing Position 0 at Time 1
Writing Position 1 at Time 1
Writing Position 2 at Time 1
Writing Position 3 at Time 1
Writing Position 4 at Time 1
Writing Position 5 at Time 1
Writing Position 6 at Time 1
Writing Position 7 at Time 1
Writing Position 8 at Time 1
Writing Position 9 at Time 1
Writing Position 0 at Time 2
Writing Position 1 at Time 2
Writing Position 2 at Time 2
Writing Position 3 at Time 2
Writing Position 4 at Time 2
Writing Position 5 at Time 2
Writing Position 6 at Time 2
Writing Position 7 at Time 2
Writing Position 8 at Time 2
Writing Position 9 at Time 2


# Writing Data with HCS Specification

##### create an HCS 4x4 Plate with rows A, B, C, D and columns 1, 2, 3, 4.  Each well then contains 2 FOV's.

In [36]:
hcs_meta = {'plate':{'acquisitions': [{'id': 1,
                     'maximumfieldcount': 2,
                     'name': 'Dataset',
                     'starttime': 0}],
  'columns': [{'name': '1'},
           {'name': '2'},
           {'name': '3'},
           {'name': '4'}],
  'field_count': 2,
  'name': 'MultiWell_Plate_Example',
  'rows': [{'name': 'A'},
          {'name': 'B'},
          {'name': 'C'},
          {'name': 'D'}],
  'version': '0.1',
  'wells': [{'path': 'A/1'},
            {'path': 'A/2'},
            {'path': 'A/3'},
            {'path': 'A/4'},
            {'path': 'B/1'},
            {'path': 'B/2'},
            {'path': 'B/3'},
            {'path': 'B/4'},
            {'path': 'C/1'},
            {'path': 'C/2'},
            {'path': 'C/3'},
            {'path': 'C/4'},
            {'path': 'D/1'},
            {'path': 'D/2'},
            {'path': 'D/3'},
            {'path': 'D/4'}]},
           
       
'well': [{'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]},
         {'images': [{'path': 'FOV1'}, {'path': 'FOV2'}]}]
    }
    

In [39]:
writer = WaveorderWriter('/Users/cameron.foltz/Desktop/Test', hcs=True, hcs_meta=hcs_meta, verbose=True)

In [40]:
writer.create_zarr_root('waveOrder_Writer_ExampleHCS.zarr')

Creating new zarr store at /Users/cameron.foltz/Desktop/Test/waveOrder_Writer_ExampleHCS.zarr


In [43]:
print(writer.store.tree())

/
 ├── A
 │   ├── 1
 │   │   ├── FOV1
 │   │   └── FOV2
 │   ├── 2
 │   │   ├── FOV1
 │   │   └── FOV2
 │   ├── 3
 │   │   ├── FOV1
 │   │   └── FOV2
 │   └── 4
 │       ├── FOV1
 │       └── FOV2
 ├── B
 │   ├── 1
 │   │   ├── FOV1
 │   │   └── FOV2
 │   ├── 2
 │   │   ├── FOV1
 │   │   └── FOV2
 │   ├── 3
 │   │   ├── FOV1
 │   │   └── FOV2
 │   └── 4
 │       ├── FOV1
 │       └── FOV2
 ├── C
 │   ├── 1
 │   │   ├── FOV1
 │   │   └── FOV2
 │   ├── 2
 │   │   ├── FOV1
 │   │   └── FOV2
 │   ├── 3
 │   │   ├── FOV1
 │   │   └── FOV2
 │   └── 4
 │       ├── FOV1
 │       └── FOV2
 └── D
     ├── 1
     │   ├── FOV1
     │   └── FOV2
     ├── 2
     │   ├── FOV1
     │   └── FOV2
     ├── 3
     │   ├── FOV1
     │   └── FOV2
     └── 4
         ├── FOV1
         └── FOV2


In [50]:
P = 32
T = 3
C = 2
Z = 11
Y = 256
X = 256

data = np.random.rand(P, T, C, Z, Y, X)

data_shape = (3,2,11,256,256)
chunk_size = (1,1,1,256,256)
chan_names = ['DAPI', 'Phase3D']
clims = [(0,1000),(-0.3, 0.3)]
dtype = 'float32'

In [51]:
# Must first initialize all of the arrays, since you will be returning to them 
# at different times in the data-saving process.
num_pos = 32

for i in range(num_pos):
    writer.open_position(i)
    writer.init_array(data_shape, chunk_size, dtype, chan_names, clims, overwrite=True)    
# Now begin the rime loop 
for t in range(3):
    
    # At each time point, write data for all positions
    for pos in range(num_pos):
        
        print(f'Writing Position {pos} at Time {t}')
        
        #change to the correct position group
        writer.open_position(pos)
        
        #write this position data
        writer.write(data[pos, t], t=t, c=[0,2], z=[0,65])

Opening subgroup A/1/FOV1
Opening subgroup A/1/FOV2
Opening subgroup A/2/FOV1
Opening subgroup A/2/FOV2
Opening subgroup A/3/FOV1
Opening subgroup A/3/FOV2
Opening subgroup A/4/FOV1
Opening subgroup A/4/FOV2
Opening subgroup B/1/FOV1
Opening subgroup B/1/FOV2
Opening subgroup B/2/FOV1
Opening subgroup B/2/FOV2
Opening subgroup B/3/FOV1
Opening subgroup B/3/FOV2
Opening subgroup B/4/FOV1
Opening subgroup B/4/FOV2
Opening subgroup C/1/FOV1
Opening subgroup C/1/FOV2
Opening subgroup C/2/FOV1
Opening subgroup C/2/FOV2
Opening subgroup C/3/FOV1
Opening subgroup C/3/FOV2
Opening subgroup C/4/FOV1
Opening subgroup C/4/FOV2
Opening subgroup D/1/FOV1
Opening subgroup D/1/FOV2
Opening subgroup D/2/FOV1
Opening subgroup D/2/FOV2
Opening subgroup D/3/FOV1
Opening subgroup D/3/FOV2
Opening subgroup D/4/FOV1
Opening subgroup D/4/FOV2
Writing Position 0 at Time 0
Opening subgroup A/1/FOV1
Writing Position 1 at Time 0
Opening subgroup A/1/FOV2
Writing Position 2 at Time 0
Opening subgroup A/2/FOV1
Wri

In [52]:
print(writer.store.tree())

/
 ├── A
 │   ├── 1
 │   │   ├── FOV1
 │   │   │   └── array (3, 2, 11, 256, 256) float32
 │   │   └── FOV2
 │   │       └── array (3, 2, 11, 256, 256) float32
 │   ├── 2
 │   │   ├── FOV1
 │   │   │   └── array (3, 2, 11, 256, 256) float32
 │   │   └── FOV2
 │   │       └── array (3, 2, 11, 256, 256) float32
 │   ├── 3
 │   │   ├── FOV1
 │   │   │   └── array (3, 2, 11, 256, 256) float32
 │   │   └── FOV2
 │   │       └── array (3, 2, 11, 256, 256) float32
 │   └── 4
 │       ├── FOV1
 │       │   └── array (3, 2, 11, 256, 256) float32
 │       └── FOV2
 │           └── array (3, 2, 11, 256, 256) float32
 ├── B
 │   ├── 1
 │   │   ├── FOV1
 │   │   │   └── array (3, 2, 11, 256, 256) float32
 │   │   └── FOV2
 │   │       └── array (3, 2, 11, 256, 256) float32
 │   ├── 2
 │   │   ├── FOV1
 │   │   │   └── array (3, 2, 11, 256, 256) float32
 │   │   └── FOV2
 │   │       └── array (3, 2, 11, 256, 256) float32
 │   ├── 3
 │   │   ├── FOV1
 │   │   │   └── array (3, 2, 11, 256, 256) float